# Exemple de base de classification avec TensorFlow

Ce carnet de notes est un compagnon de [Un guide visuel et interactif des bases des réseaux neuronaux](https://rr0.org/people/a/AlammarJay/visual-interactive-guide-basics-neural-networks/index_fr.html).

Voici un exemple de comment faire un classement sur un ensemble de données simple dans TensorFlow.
Pour résumer, on construit un modèle pour aider une amie à choisir une maison à acheter.
Elle nous a donné le tableau ci-dessous de maisons et si elle les aime ou non.
Nous allons élaborer un modèle qui prend une surface de maison et un nombre de salles de bain en entrée,
et sort une prédiction indiquant si elle aimera la maison ou non.

| Surface (pieds²) (x1) | Salles de bain (x2) | Libellé (y) |
 | --- | --- | --- |
 | 2104 |  3 | Bon |
 | 1600 |  3 | Bon |
 | 2400 |  3 | Bon |
 | 1416 | 	2 | Mauvais |
 | 3000 | 	4 | Mauvais |
 | 1985 | 	4 | Bon |
 | 1534 | 	3 | Mauvais |
 | 1427 | 	3 | Bon |
 | 1380 | 	3 | Bon |
 | 1494 | 	3 | Bon |

Nous allons commencer en chargeant nos bibliothèques favorites

In [1]:
%matplotlib inline               
import pandas as pd              # Une belle bibliothèque pour nous aider à travailler les données sous forme tabulaire
import numpy as np               # Pour que l'on puisse utiliser les matrices de nombres. Pandas comme TensorFlow en ont besoin.
import matplotlib.pyplot as plt  # Visualiser les choses
import tensorflow as tf          # Le feu des dieux


Nous allons ensuite charger le CSV des données de maisons.
Pandas est une bibliothèque incroyable offrant une grande souplesse dans le traitement des données tabulaires.
On charge les tableaux (ou des fichiers csv, ou des feuilles excel) dans une "data frame", et on les traite comme on veut.
Vous pouvez voir ça comme une manière programmatique de faire beaucoup des choses que vous avez faites avant avec Excel.

In [2]:
dataframe = pd.read_csv("data.csv") # Demandons à Pandas de charger notre ensemble de données comme une dataframe
dataframe = dataframe.drop(["index", "price", "sq_price"], axis=1) # Enlever les colonnes qui ne nous intéressent pas
dataframe = dataframe[0:10] # Nous n'allons utiliser que les 10 premières lignes de l'ensemble de données dans cet exemple
dataframe # Demandons au carnet de nous montrer à quoi ressemble le dataframe maintenant

,area,bathrooms
0,2104.0,3.0
1,1600.0,3.0
2,2400.0,3.0
3,1416.0,2.0
4,3000.0,4.0
5,1985.0,4.0
6,1534.0,3.0
7,1427.0,3.0
8,1380.0,3.0
9,1494.0,3.0


La dataframe n'a maintenant que les caractéristiques. Ajoutons les libellés.

In [3]:
dataframe.loc[:, ("y1")] = [1, 1, 1, 0, 0, 1, 0, 1, 1, 1] # Ceci est la liste de notre amie des maisons qu'elle a aimées
                                                          # 1 = bon, 0 = mauvais
dataframe.loc[:, ("y2")] = dataframe["y1"] == 0           # y2 est la négation de y1
dataframe.loc[:, ("y2")] = dataframe["y2"].astype(int)    # Transforme les valeurs TRUE/FALSE en 1/0
# y2 signifie que nous n'aimons pas une maison
# (Oui, c'est redondant. Mais apprendre à le faire de cette manière ouvre la porte au classement multi-classes)
dataframe # À quoi ressemble notre dataframe maintenant ?

,area,bathrooms,y1,y2
0,2104.0,3.0,1,0
1,1600.0,3.0,1,0
2,2400.0,3.0,1,0
3,1416.0,2.0,0,1
4,3000.0,4.0,0,1
5,1985.0,4.0,1,0
6,1534.0,3.0,0,1
7,1427.0,3.0,1,0
8,1380.0,3.0,1,0
9,1494.0,3.0,1,0


Maintenant que nous avons toutes nos données dans la dataframe,
nous devons les arranger sous forme de matrices avant de les donner à TensorFlow

In [4]:
inputX = dataframe.loc[:, ['area', 'bathrooms']].as_matrix()
inputY = dataframe.loc[:, ["y1", "y2"]].as_matrix()

/Users/javarome/anaconda3/envs/zelros-cs-nlp/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/Users/javarome/anaconda3/envs/zelros-cs-nlp/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


Maintenant notre matrice d'entrée ressemble à ceci:

In [5]:
inputX

array([[2.104e+03, 3.000e+00],
       [1.600e+03, 3.000e+00],
       [2.400e+03, 3.000e+00],
       [1.416e+03, 2.000e+00],
       [3.000e+03, 4.000e+00],
       [1.985e+03, 4.000e+00],
       [1.534e+03, 3.000e+00],
       [1.427e+03, 3.000e+00],
       [1.380e+03, 3.000e+00],
       [1.494e+03, 3.000e+00]])

Et notre matrice de libellés à ceci :

In [6]:
inputY

array([[1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0]])

Préparons des paramètres pour le processus d'entraînement

In [7]:
# Paramètres
learning_rate = 0.000001
training_epochs = 2000
display_step = 50
n_samples = inputY.size

Et maintenant définissons les opérations TensorFlow.
Notez qu'il s'agit d'une étape de déclaration où nous indiquons à TensorFlow comment est calculée la prédiction.
Si nous l'exécutons, aucun calcul ne sera effectué.
Il répondra juste qu'il sait maintenant comment faire l'opération.

In [8]:
x = tf.placeholder(tf.float32, [None, 2])   # Ok TensorFlow, on te donnera un tableau d'exemples. Chaque exemple sera
                                            # un tableau de 2 valeurs décimales (surface, et nombre de salles de bain).
                                            # "None" veut dire qu'on peut te donner n'importe quel nombre d'exemples
                                            # Notez que nous ne lui avons pas encore fourni les valeurs
            
W = tf.Variable(tf.zeros([2, 2]))           # Maintiens un matrice de nombre décimaux de 2 x 2 pour les poids que nous mettrons à jour constamment
                                            # lors du processus d'entraînement (on les met tous à zéro pour commencer)
    
b = tf.Variable(tf.zeros([2]))              # Maintiens aussi 2 valeurs de biais

y_values = tf.add(tf.matmul(x, W), b)       # La 1ère étape pour calculer la prédiction sera de multiplier
                                            # la matrice des entrées par la matrice des poids puis d'ajouter les biais
    
y = tf.nn.softmax(y_values)                 # Puis utiliser le softmax comme "fonction d'activation" qui traduit les
                                            # nombres en sortie de la couche précédente sous forme de probabilités
    
y_ = tf.placeholder(tf.float32, [None,2])   # Pour l'entraînement, on te fournira aussi une matrice de libellés

Spécifions notre fonction de coût et utilisons la Descente de Gradient

In [9]:

# Fonction de coût : Erreur quadratique moyenne
cost = tf.reduce_sum(tf.pow(y_ - y, 2))/(2*n_samples)
# Descente de gradient
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [10]:
# Initialise les variables et la session tensorflow
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


*Roulement de tambours*

Et maintenant l'entraînement à proprement parler

In [11]:
for i in range(training_epochs):  
    sess.run(optimizer, feed_dict={x: inputX, y_: inputY}) # Itère dans la descente de gradient en utilisant nos entrées et libellés

    # C'est tout ! Le reste de la cellule ne fait qu'afficher des messages de deboguage.
    # Affiche les logs par étape d'époque
    if (i) % display_step == 0:
        cc = sess.run(cost, feed_dict={x: inputX, y_:inputY})
        print("Etape d entraînement :", '%04d' % (i), "coût=", "{:.9f}".format(cc)) #, \"W=", sess.run(W), "b=", sess.run(b)

print("Optimisation terminée !")
training_cost = sess.run(cost, feed_dict={x: inputX, y_: inputY})
print("Coût d'entraînement=", training_cost, "W=", sess.run(W), "b=", sess.run(b), '\n')


Etape d entraînement : 0000 coût= 0.114958666
Etape d entraînement : 0050 coût= 0.109539941
Etape d entraînement : 0100 coût= 0.109539881
Etape d entraînement : 0150 coût= 0.109539807
Etape d entraînement : 0200 coût= 0.109539740
Etape d entraînement : 0250 coût= 0.109539680
Etape d entraînement : 0300 coût= 0.109539606
Etape d entraînement : 0350 coût= 0.109539531
Etape d entraînement : 0400 coût= 0.109539464
Etape d entraînement : 0450 coût= 0.109539390
Etape d entraînement : 0500 coût= 0.109539323
Etape d entraînement : 0550 coût= 0.109539248
Etape d entraînement : 0600 coût= 0.109539203
Etape d entraînement : 0650 coût= 0.109539129
Etape d entraînement : 0700 coût= 0.109539054
Etape d entraînement : 0750 coût= 0.109538987
Etape d entraînement : 0800 coût= 0.109538913
Etape d entraînement : 0850 coût= 0.109538853
Etape d entraînement : 0900 coût= 0.109538786
Etape d entraînement : 0950 coût= 0.109538712
Etape d entraînement : 1000 coût= 0.109538652
Etape d entraînement : 1050 coût= 

L'entraînement est maintenant terminé.
TensorFlow garde maintenant notre modèle entraîné
(qui se résume aux opérations définies,
plus les variables W et b qui ont été trouvées par le processus d'entraînement).

Une valeur de coût de 0.109537 est-elle bonne ou mauvaise ? Aucune idée.
Au moins c'est mieux que la 1ère valeur de coût de 0.114958666.
Utilisons le modèle sur notre dataset pour voir comment il se comporte, quand même :

In [12]:
sess.run(y, feed_dict={x: inputX })

array([[0.7112522 , 0.28874776],
       [0.66498977, 0.33501023],
       [0.73657656, 0.26342347],
       [0.6471879 , 0.3528121 ],
       [0.78335613, 0.2166439 ],
       [0.7006948 , 0.29930523],
       [0.6586633 , 0.34133676],
       [0.6482863 , 0.35171375],
       [0.6436828 , 0.35631716],
       [0.65480113, 0.3451989 ]], dtype=float32)

Donc il pense que toutes sont des bonnes maisons.
Ca fait 7 bonnes réponses sur 10. Pas super impressionnant.
Un modèle avec une couche cachée devrait être meilleur, je suppose.

En passant, voici comment j'ai calculé les valeurs de softmax dans l'article :

In [13]:
sess.run(tf.nn.softmax([1., 2.]))

array([0.26894143, 0.7310586 ], dtype=float32)